In [1]:
import pandas as pd
import numpy as np
import csv

### Data Cleaning

- This notebook takes the file created from the [processed_data.ipynb](https://github.com/biglocalnews/WARN_Cali_analysis/blob/master/notebooks/processed_data.ipynb) and cleans it. This process involves removing whitespace from key columns, resolving conflicts between incorrect company names and removing punctuation from the company column. 
- This notebook also adds population figures for each county to our layoffs notices data.
- Finally, the data gets exported to the `open_refine` directory which leads to a manual cleaning and standardization of company names with [OpenRefine](https://openrefine.org/) software.

In [2]:
# Here we read in the necessary files from the cleaning directory

cali_layoffs = pd.read_csv('../data/cleaning/california_warn_raw_recent.csv')
cali_population = pd.read_csv('../data/cleaning/county_population.csv', delimiter='\t', thousands=',')

In [3]:
# Calling the function .head() on our dataframe allows us to see what's in the data

cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent


### Clean County, Company and City Columns

In [4]:
# Removal of whitespace, and hidden characters

cali_layoffs['Company'] = cali_layoffs['Company'].str.replace('\\r', '')
cali_layoffs['Company'] = cali_layoffs['Company'].str.replace('\\n', '')
cali_layoffs['City'] = cali_layoffs['City'].str.replace('\\r', '')
cali_layoffs['City'] = cali_layoffs['City'].str.replace('\\n', '')
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('\\r', '')
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('\\n', '')

In [5]:
# Removal of punctuation

cali_layoffs['Company 2'] = cali_layoffs['Company']
cali_layoffs['Company 2'] = cali_layoffs['Company 2'].str.upper()
cali_layoffs['Company 2'] = cali_layoffs['Company 2'].str.replace("[,.\\-\\';:*#&\\(\\)\\/\\_\\@]",'')
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Company 2
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,BAY CLUB REDONDO BEACH
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,BAY CLUB ROLLING HILLS
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,BAY CLUB SANTA MONICA
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,WEBER METALS INC
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,STONETREE GOLF CLUB


In [6]:
# stripping leading and trailing whitespace for a column we will work on

cali_layoffs['County'] = cali_layoffs['County'].str.strip()
cali_layoffs['County Orig'] = cali_layoffs['County']
cali_layoffs['County'].fillna('Not Available', inplace=True)

In [7]:
cali_layoffs.loc[~(cali_layoffs['County'].str.endswith('County')) & (cali_layoffs['County'] != 'Not Available'),'County'] += " County"
cali_layoffs['County'].unique()

array(['Los Angeles County', 'Marin County', 'Orange County',
       'Sacramento County', 'San Bernardino County',
       'San Francisco County', 'San Mateo County', 'Santa Clara County',
       'Ventura County', 'Kern County', 'Stanislaus County',
       'Contra Costa County', 'Alameda County', 'San Diego County',
       'Kings County', 'San Joaquin County', 'Sonoma County',
       'Tulare County', 'Napa County', 'Monterey County', 'Placer County',
       'Mono County', 'Riverside County', 'Fresno County',
       'Mendocino County', 'Del Norte County', 'Yolo County',
       'San Luis Obispo County', 'Madera County', 'Shasta County',
       'Solano County', 'Santa Barbara County', 'Imperial County',
       'Mariposa County', 'Santa Cruz County', 'El Dorado County',
       'Butte County', 'Sutter County', 'Yuba County', 'Siskiyou County',
       'Calaveras County', 'Tehama County', 'Glenn County', 'Inyo County',
       'Not Available', 'San Benito County', 'Merced County',
       'Nevad

In [8]:
# Creating a year column based on the last 4 characters of the Notice Date

cali_layoffs['Year'] = cali_layoffs['Notice Date'].str[-4:]
cali_layoffs['Year'] = cali_layoffs['Year'].astype(np.int64)
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Company 2,County Orig,Year
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,BAY CLUB REDONDO BEACH,Los Angeles County,2020
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,BAY CLUB ROLLING HILLS,Los Angeles County,2020
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,BAY CLUB SANTA MONICA,Los Angeles County,2020
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,WEBER METALS INC,Los Angeles County,2020
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,STONETREE GOLF CLUB,Marin County,2020


In [9]:
# Viewing unique County Names

cali_layoffs['County'].unique()

array(['Los Angeles County', 'Marin County', 'Orange County',
       'Sacramento County', 'San Bernardino County',
       'San Francisco County', 'San Mateo County', 'Santa Clara County',
       'Ventura County', 'Kern County', 'Stanislaus County',
       'Contra Costa County', 'Alameda County', 'San Diego County',
       'Kings County', 'San Joaquin County', 'Sonoma County',
       'Tulare County', 'Napa County', 'Monterey County', 'Placer County',
       'Mono County', 'Riverside County', 'Fresno County',
       'Mendocino County', 'Del Norte County', 'Yolo County',
       'San Luis Obispo County', 'Madera County', 'Shasta County',
       'Solano County', 'Santa Barbara County', 'Imperial County',
       'Mariposa County', 'Santa Cruz County', 'El Dorado County',
       'Butte County', 'Sutter County', 'Yuba County', 'Siskiyou County',
       'Calaveras County', 'Tehama County', 'Glenn County', 'Inyo County',
       'Not Available', 'San Benito County', 'Merced County',
       'Nevad

### Dropping 'Cancelled' Layoffs

In [10]:
# There are records where it says 'cancelled' in the name - we assume these layoffs were cancelled, thus we are dropping those records from our data
# This first 2 lines just identify which records have the words 'cancelled' in them

cancelled_layoffs = cali_layoffs[cali_layoffs['Company 2'].str.contains('CANCELLED')]
len(cancelled_layoffs)

10

In [11]:
# View the records we are dropping

cancelled_layoffs

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Company 2,County Orig,Year
1082,07/17/2020,09/23/2020,10/02/2020,**JC Penney (Cancelled),San Bernardino,San Bernardino County,109.0,Closure Permanent,JC PENNEY CANCELLED,San Bernardino County,2020
1085,07/01/2020,09/04/2020,10/02/2020,"**CoreLogic Credco, LLC (Cancelled)",San Diego,San Diego County,137.0,Layoff Permanent,CORELOGIC CREDCO LLC CANCELLED,San Diego County,2020
4449,04/27/2015,06/26/2015,04/27/2015,"Health Net, Inc. (CANCELLED)**",Woodland Hills,Not Available,1217.0,Layoff Unknown at this time,HEALTH NET INC CANCELLED,NaN,2015
4458,04/27/2015,06/26/2015,04/28/2015,"Health Net, Inc. (CANCELLED)**",Rancho Cordova,Not Available,385.0,Layoff Unknown at this time,HEALTH NET INC CANCELLED,NaN,2015
4459,04/27/2015,06/26/2015,04/28/2015,"Health Net, Inc. (CANCELLED)**",Rancho Cordova,Not Available,520.0,Layoff Unknown at this time,HEALTH NET INC CANCELLED,NaN,2015
6137,04/28/2020,06/29/2020,05/04/2020,Public Health Institute (CANCELLED)**,Oakland,Alameda County,84.0,Layoff Temporary,PUBLIC HEALTH INSTITUTE CANCELLED,Alameda County,2020
8965,12/11/2019,01/31/2020,12/12/2019,Abercrombie & Fitch ***(CANCELLED),Palo Alto,Santa Clara County,45.0,Closure Permanent,ABERCROMBIE FITCH CANCELLED,Santa Clara County,2019
9165,08/13/2019,10/12/2019,08/20/2019,LA Hearth LLC (CANCELLED)**,Los Angeles,Los Angeles County,73.0,Closure Permanent,LA HEARTH LLC CANCELLED,Los Angeles County,2019
9166,07/29/2019,09/30/2019,08/19/2019,AECOM (CANCELLED)**,San Diego,San Diego County,3.0,Layoff Permanent,AECOM CANCELLED,San Diego County,2019
9297,08/19/2016,10/23/2017,08/22/2017,Chili's Grill & Bar Restaurant (CANCELLED)**,Thousand Oaks,Ventura County,48.0,Closure Permanent,CHILIS GRILL BAR RESTAURANT CANCELLED,Ventura County,2016


In [12]:
# Get the specific index of those records

cancelled_layoffs.index

Int64Index([1082, 1085, 4449, 4458, 4459, 6137, 8965, 9165, 9166, 9297], dtype='int64')

In [13]:
# Get the length of our dataframe

len(cali_layoffs)

9864

In [14]:
# Drop cancelled records and then re-calculate len of our dataframe

cali_layoffs.drop(cancelled_layoffs.index, inplace=True)
len(cali_layoffs)

9854

### Cleaning Layoff Types

Type Unknown and unknown at this time will be combined.

In [15]:
# View unqiue number of options for the Layoff/Closure colum

cali_layoffs['Layoff/Closure'].unique()

array(['Layoff Permanent', 'Layoff Type Unknown', 'Layoff Temporary',
       'Closure Temporary', 'Closure Permanent', 'Closure Type Unknown',
       'Layoff Unknown at this time', 'Closure Unknown at this time',
       'Closure Unknown at thistime', 'Layoff Unknown at thistime'],
      dtype=object)

In [16]:
# Standardize the columns by stripping whitespace, making everything lowercase and replacing unknown with uncategorized, replacing mispellings 

cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.strip()
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.lower()
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.replace('unknown at this time', 'type unknown')
cali_layoffs['Layoff/Closure clean'] = cali_layoffs['Layoff/Closure clean'].str.replace('type unknown', 'type uncategorized')
cali_layoffs['Layoff/Closure clean'].unique()

array(['layoff permanent', 'layoff type uncategorized',
       'layoff temporary', 'closure temporary', 'closure permanent',
       'closure type uncategorized'], dtype=object)

In [17]:
# Taking another look at our data because we've made a few changes

cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Company 2,County Orig,Year,Layoff/Closure clean
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,BAY CLUB REDONDO BEACH,Los Angeles County,2020,layoff permanent
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,BAY CLUB ROLLING HILLS,Los Angeles County,2020,layoff permanent
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,BAY CLUB SANTA MONICA,Los Angeles County,2020,layoff permanent
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,WEBER METALS INC,Los Angeles County,2020,layoff permanent
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,STONETREE GOLF CLUB,Marin County,2020,layoff permanent


### Ambiguous Cities

In doing some preliminary exploration of the data, we learned that some cities have been assigned the incorrect county name. The code below finds the cities with more than one county assigned. This code identifies cities that have more than one county paired with it. It then creates a list of tuples with the city name and the two counties it got paired with.

After that, we select the correct county manually and create a dictionary with the correct city,county key,value pairs. We then take that dictionary and go through our dataframe to find the entries that do not match this pairing. When a record that doesn't match is found, the record is updated according to the key,value pair in our dictionary.

The code also populates empty counties with the correct county name based on the name of the city.

This leaves us with an updated dataset where all the cities have the correct counties listed in the `County` column.
In order to see the original incorrect entries, the `County Orig` column has the original untouched data.

In [18]:
# Make my dataframe a dictionary

cali_dict = cali_layoffs.to_dict(orient='records')
cali_dict[0:2]

[{'Notice Date': '06/09/2020',
  'Effective Date': '06/07/2020',
  'Received Date': '07/01/2020',
  'Company': 'Bay Club Redondo Beach',
  'City': 'Redondo Beach',
  'County': 'Los Angeles County',
  'Employees': 102.0,
  'Layoff/Closure': 'Layoff Permanent',
  'Company 2': 'BAY CLUB REDONDO BEACH',
  'County Orig': 'Los Angeles County',
  'Year': 2020,
  'Layoff/Closure clean': 'layoff permanent'},
 {'Notice Date': '06/09/2020',
  'Effective Date': '06/07/2020',
  'Received Date': '07/01/2020',
  'Company': 'Bay Club Rolling Hills',
  'City': 'Rolling Hills Estates',
  'County': 'Los Angeles County',
  'Employees': 64.0,
  'Layoff/Closure': 'Layoff Permanent',
  'Company 2': 'BAY CLUB ROLLING HILLS',
  'County Orig': 'Los Angeles County',
  'Year': 2020,
  'Layoff/Closure clean': 'layoff permanent'}]

In [19]:
# List unqiue counties

cali_layoffs['County'].unique()

array(['Los Angeles County', 'Marin County', 'Orange County',
       'Sacramento County', 'San Bernardino County',
       'San Francisco County', 'San Mateo County', 'Santa Clara County',
       'Ventura County', 'Kern County', 'Stanislaus County',
       'Contra Costa County', 'Alameda County', 'San Diego County',
       'Kings County', 'San Joaquin County', 'Sonoma County',
       'Tulare County', 'Napa County', 'Monterey County', 'Placer County',
       'Mono County', 'Riverside County', 'Fresno County',
       'Mendocino County', 'Del Norte County', 'Yolo County',
       'San Luis Obispo County', 'Madera County', 'Shasta County',
       'Solano County', 'Santa Barbara County', 'Imperial County',
       'Mariposa County', 'Santa Cruz County', 'El Dorado County',
       'Butte County', 'Sutter County', 'Yuba County', 'Siskiyou County',
       'Calaveras County', 'Tehama County', 'Glenn County', 'Inyo County',
       'Not Available', 'San Benito County', 'Merced County',
       'Nevad

In [20]:
# Create a list of records where the county column has string 'Not Available'

rows_with_no_county = [r for r in cali_dict if r["County"] == "Not Available"]
len(rows_with_no_county)

2114

In [21]:
# Shows a single record

rows_with_no_county[0]

{'Notice Date': '06/22/2015',
 'Effective Date': '03/25/2016',
 'Received Date': '07/01/2015',
 'Company': 'Maxim Integrated Product',
 'City': 'San Jose',
 'County': 'Not Available',
 'Employees': 150.0,
 'Layoff/Closure': 'Closure Permanent',
 'Company 2': 'MAXIM INTEGRATED PRODUCT',
 'County Orig': nan,
 'Year': 2015,
 'Layoff/Closure clean': 'closure permanent'}

In [22]:
# Create a list of records where we get # of cities that have no county listed

cities = [r["City"] for r in rows_with_no_county]

In [23]:
# Get length of that list

len(cities)

2114

In [24]:
# creates a set out of the cities list

cities = set(cities)

In [25]:
# Get length of that list

len(cities)

355

In [26]:
# Create a list of records where we get # of cities that have a county listed

rows_with_county = [r for r in cali_dict if r["County"] != "Not Available"]

In [27]:
# Get length of list we created above

len(rows_with_county)

7740

In [28]:
# Take a look at a specific record within the above list

rows_with_county[334]

{'Notice Date': '07/03/2020',
 'Effective Date': '07/03/2020',
 'Received Date': '08/03/2020',
 'Company': 'Wyndham San Diego Bayside',
 'City': 'San Diego',
 'County': 'San Diego County',
 'Employees': 88.0,
 'Layoff/Closure': 'Layoff Temporary',
 'Company 2': 'WYNDHAM SAN DIEGO BAYSIDE',
 'County Orig': 'San Diego County',
 'Year': 2020,
 'Layoff/Closure clean': 'layoff temporary'}

In [29]:
# Creates a dictionary, loops through rows_with_county list
# Every city gets added as a key with its given county as the value
# If a city has two counties, the entry will have two values

city_to_counties = {}
for row in rows_with_county:
    city = row["City"]
    county = row["County"]
    if city not in city_to_counties:
        city_to_counties[city] = set()
    city_to_counties[city].add(county)

In [30]:
# Makes a list from cities that only have one value

unambiguous_cities = [pair for pair in city_to_counties.items() if len(pair[1]) == 1]

In [31]:
# Gets length of above list

len(unambiguous_cities)

525

In [32]:
# Makes a dictionary from cities that only have one value

unambiguous_cities = {pair[0]: list(pair[1])[0] for pair in city_to_counties.items() if len(pair[1]) == 1}

In [33]:
# Loop through rows_with_no_county list
# Adds the correct county based on unambiguous_cities dictionary which has the correct city, county pairings

for row in rows_with_no_county:
    if row["City"] in unambiguous_cities:
        row["County"] = unambiguous_cities[row["City"]]

In [34]:
# Take a look at a random record

rows_with_no_county[:2]

[{'Notice Date': '06/22/2015',
  'Effective Date': '03/25/2016',
  'Received Date': '07/01/2015',
  'Company': 'Maxim Integrated Product',
  'City': 'San Jose',
  'County': 'Santa Clara County',
  'Employees': 150.0,
  'Layoff/Closure': 'Closure Permanent',
  'Company 2': 'MAXIM INTEGRATED PRODUCT',
  'County Orig': nan,
  'Year': 2015,
  'Layoff/Closure clean': 'closure permanent'},
 {'Notice Date': '06/30/2015',
  'Effective Date': '08/29/2015',
  'Received Date': '07/01/2015',
  'Company': 'McGraw-Hill Education',
  'City': 'Monterey',
  'County': 'Monterey County',
  'Employees': 137.0,
  'Layoff/Closure': 'Layoff Unknown at this time',
  'Company 2': 'MCGRAWHILL EDUCATION',
  'County Orig': nan,
  'Year': 2015,
  'Layoff/Closure clean': 'layoff type uncategorized'}]

In [35]:
# Makes list from cities that have more than one value

ambiguous_cities = [pair for pair in city_to_counties.items() if len(pair[1]) > 1]

In [36]:
# shows the list of cities with more than one county listed

ambiguous_cities

[('Sacramento', {'Del Norte County', 'Sacramento County'}),
 ('San Francisco', {'San Francisco County', 'San Mateo County'}),
 ('Walnut Creek', {'Alameda County', 'Contra Costa County'}),
 ('Los Angeles', {'Alameda County', 'Los Angeles County'}),
 ('Hayward', {'Alameda County', 'Contra Costa County'}),
 ('City of Industry', {'Los Angeles County', 'Orange County'}),
 ('Roseville', {'Placer County', 'Sacramento County'}),
 ('Berkeley', {'Alameda County', 'Contra Costa County'}),
 ('Cerritos', {'Los Angeles County', 'Orange County'}),
 ('Vista', {'San Diego County', 'San Francisco County'}),
 ('Del Mar', {'Orange County', 'San Diego County'}),
 ('Indian Wells', {'Riverside County', 'San Benito County'}),
 ('Rancho', {'Los Angeles County', 'San Bernardino County'}),
 ('Los Gatos', {'Santa Clara County', 'Santa Cruz County'})]

### Manual research narrowed down which cities actually belonged to which counties
resulting in our updated `ambiguous_cities` dictionary. `Rancho` is the only city on the list where the two counties actually have cities with the same name. It will be left off the list. 

***The dictionary below may need further manual adjusting if this notebook is being run with updated data.***

In [37]:
dict_cities = {
    'Sacramento':'Sacramento County',
    'San Francisco':'San Francisco County',
    'Hayward':'Alameda County',
    'Indian Wells':'Riverside County',
    'Cerritos':'Los Angeles County',
    'Vista':'San Diego County',
    'Roseville':'Placer County',
    'City of Industry':'Los Angeles County',
    'Del Mar':'San Diego County',
    'Walnut Creek':'Contra Costa County',
    'Los Gatos':'Santa Clara County',
    'Los Angeles':'Los Angeles County',
    'Berkeley': 'Alameda County'
}

### Clean County Names
Below is our code to correct County Names based on City Names. 
We are taking the dictionary above, looping through the city and county names, which are the keys and values, replacing the incorrect County Name with the correct one for each city's record.

In [38]:
for city, county in dict_cities.items():
    cali_layoffs.loc[cali_layoffs['City'] == city, ['County']] = county

In [39]:
# Take a look at a city - compare the County column with the County Orig column.
# The County column should have the correct county for each entry
# The County Orig column may have some missing values and/or incorrect county names

los_gatos_clean = cali_layoffs[cali_layoffs['City'] == 'Los Gatos']
los_gatos_clean.head(8)

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Company 2,County Orig,Year,Layoff/Closure clean
3060,04/26/2019,06/30/2019,04/26/2019,Covia Communities,Los Gatos,Santa Clara County,66.0,Closure Permanent,COVIA COMMUNITIES,Santa Clara,2019,closure permanent
3173,06/03/2019,10/01/2019,06/04/2019,"Roche Molecular Systems, Inc.",Los Gatos,Santa Clara County,17.0,Closure Permanent,ROCHE MOLECULAR SYSTEMS INC,Santa Clara,2019,closure permanent
3327,09/01/2016,11/01/2016,09/02/2016,Valley Medical Oncology Consultants,Los Gatos,Santa Clara County,29.0,Layoff Permanent,VALLEY MEDICAL ONCOLOGY CONSULTANTS,NaN,2016,layoff permanent
4043,10/01/2014,12/31/2014,10/16/2014,Prometheus Real Estate Group,Los Gatos,Santa Clara County,8.0,Layoff Permanent,PROMETHEUS REAL ESTATE GROUP,NaN,2014,layoff permanent
4406,03/27/2015,03/16/2015,03/27/2015,"Cryptic Studios, Inc.",Los Gatos,Santa Clara County,27.0,Layoff Permanent,CRYPTIC STUDIOS INC,NaN,2015,layoff permanent
4658,06/23/2020,08/20/2020,06/29/2020,Boston Scientific Corporation,Los Gatos,Santa Clara County,60.0,Closure Permanent,BOSTON SCIENTIFIC CORPORATION,Santa Clara County,2020,closure permanent
4750,06/09/2020,06/07/2020,06/24/2020,Bay Club Courtside,Los Gatos,Santa Clara County,294.0,Layoff Permanent,BAY CLUB COURTSIDE,Santa Clara County,2020,layoff permanent
5656,03/30/2020,03/17/2020,05/20/2020,BACK OF THE HOUSE DBA SUPER DUPER,Los Gatos,Santa Clara County,30.0,Layoff Temporary,BACK OF THE HOUSE DBA SUPER DUPER,Santa Clara County,2020,layoff temporary


### Adding Population Data

The data has been grabbed from the census link above. These figures will be used to calculate per capita layoff numbers per county.

In [40]:
# Here we rename columns
# We do some data cleaning on the County Names column which has an extra character at the beginning
# We split on the comma and keep the first part of the entry because we already know it's California data

cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
cali_population['County Names'] = cali_population['Counties'].str[1:]
cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
cali_population.head(4)

,Counties,Population,County Names
0,".Alameda County, California",1671329,Alameda County
1,".Alpine County, California",1129,Alpine County
2,".Amador County, California",39752,Amador County
3,".Butte County, California",219186,Butte County


In [41]:
# Here the layoffs and population data gets merged
# Duplicate columns such as Counties are dropped
# city 2 is created as a lowercase version of the original city column

cali_merge = pd.merge(cali_layoffs, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge.drop(columns='Counties', inplace=True)
cali_merge['City 2'] = cali_merge['City'].str.lower()
cali_merge.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Company 2,County Orig,Year,Layoff/Closure clean,Population,County Names,City 2
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,BAY CLUB REDONDO BEACH,Los Angeles County,2020,layoff permanent,10039107.0,Los Angeles County,redondo beach
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,BAY CLUB ROLLING HILLS,Los Angeles County,2020,layoff permanent,10039107.0,Los Angeles County,rolling hills estates
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,BAY CLUB SANTA MONICA,Los Angeles County,2020,layoff permanent,10039107.0,Los Angeles County,santa monica
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,WEBER METALS INC,Los Angeles County,2020,layoff permanent,10039107.0,Los Angeles County,paramount
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,STONETREE GOLF CLUB,Marin County,2020,layoff permanent,258826.0,Marin County,novato


### Extracting Only Necessary Records

Now that the data has been transformed to a more usable version, we will be selecting the records that have a notice date from the year 2020 or 2019. This will allow for comparisons between a pandemic and non-pandemic year in terms of layoff notices. 

In [42]:
# Creates a list of the 2 years we will be analyzing
# Creates a new dataframe from just the records that fall within the range of the two years listed above

years = ['2020', '2019']

cali_2020_2019 = cali_merge[cali_merge["Year"].isin(years)]
cali_2020_2019.drop(columns='County Names', inplace=True)
cali_2020_2019.head()

/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/frame.py:4162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Company 2,County Orig,Year,Layoff/Closure clean,Population,City 2
0,06/09/2020,06/07/2020,07/01/2020,Bay Club Redondo Beach,Redondo Beach,Los Angeles County,102.0,Layoff Permanent,BAY CLUB REDONDO BEACH,Los Angeles County,2020,layoff permanent,10039107.0,redondo beach
1,06/09/2020,06/07/2020,07/01/2020,Bay Club Rolling Hills,Rolling Hills Estates,Los Angeles County,64.0,Layoff Permanent,BAY CLUB ROLLING HILLS,Los Angeles County,2020,layoff permanent,10039107.0,rolling hills estates
2,06/09/2020,06/07/2020,07/01/2020,Bay Club Santa Monica,Santa Monica,Los Angeles County,82.0,Layoff Permanent,BAY CLUB SANTA MONICA,Los Angeles County,2020,layoff permanent,10039107.0,santa monica
3,06/19/2020,08/21/2020,07/01/2020,"Weber Metals, Inc",Paramount,Los Angeles County,169.0,Layoff Permanent,WEBER METALS INC,Los Angeles County,2020,layoff permanent,10039107.0,paramount
4,06/09/2020,06/07/2020,07/01/2020,StoneTree Golf Club,Novato,Marin County,32.0,Layoff Permanent,STONETREE GOLF CLUB,Marin County,2020,layoff permanent,258826.0,novato


In [43]:
# Get length of the new dataset

len(cali_2020_2019)

6708

### Export to Open Refine

Now that the data has gone through some necessary transformations, we will export the file to OpenRefine. [OpenRefine](https://openrefine.org/) is software for standardizing and further cleaning data. This is a manual process which will be explained in the [open_refine_dedupe.ipynb](https://github.com/biglocalnews/WARN_Cali_analysis/blob/master/notebooks/open_refine_dedupe.ipynb). 

In [44]:
cali_2020_2019.to_csv('../data/open_refine/clean_warn_data.csv', index=False)